# Assignment 1
## Shuai Shao

In [1]:
from feat import Detector
from feat.plotting import imshow
from IPython.display import Image
import pandas as pd
import cv2
from pathlib import Path

In [33]:
df = pd.read_csv("../dataset/annotations.csv")
#df["path"] = list(map(lambda x: x.join(".jpg"), list(df.file)))
print(df)

                file   valence
0          thumbs-up  positive
1            sad-man  negative
2              claws  negative
3    laughing-couple  positive
4         by-the-sea  negative
5            arguing  negative
6   enjoying-the-sun  positive
7          piggyback  positive
8           back-off  negative
9          handshake  positive
10            tablet  negative
11         happy-man  positive
12         sad-woman  negative
13          business  positive
14              pain  negative
15          students  positive
16             bills  negative
17           bullied  negative
18      disagreement  negative
19      grandparents  positive


In [8]:
detector = Detector(face_model="retinaface", au_model = "xgb", emotion_model="resmasknet")






In [38]:

all_aus = pd.DataFrame()



def proc_image(path, detector, all_aus):

    pred = detector.detect_image(path)

    aus = pred.aus

    name = Path(path).stem
    aus['file'] = name
    aus['face'] = range(len(aus))

    if len(all_aus) < 1:
        all_aus = aus
    else:
        all_aus.append(aus)

    print(pd.DataFrame(aus))

    fb = pred.faceboxes

path="../dataset/images/back-off.jpg"
proc_image(path, detector, all_aus)
for image in df['file']:
    print(image)

print(list(map(lambda y: proc_image(y, detector, all_aus), map(lambda x: f"../dataset/images/{x}.jpg", df['file']))))


#aus.to_csv("processed/" + name + ".csv", index_label = 'face')


100%|██████████| 1/1 [00:02<00:00,  2.30s/it]
C:\Users\Shuai Shao\AppData\Local\Temp\ipykernel_22600\2147124916.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aus['file'] = name
C:\Users\Shuai Shao\AppData\Local\Temp\ipykernel_22600\2147124916.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aus['face'] = range(len(aus))


       AU01     AU02     AU04      AU05      AU06  AU07      AU09      AU10  \
0  0.517492  0.22733  0.24347  0.725215  0.070471   0.0  0.133483  0.004141   

   AU11      AU12  ...      AU17  AU20      AU23      AU24     AU25      AU26  \
0   1.0  0.079265  ...  0.283976   1.0  0.329292  0.070884  0.99722  0.462353   

       AU28      AU43      file  face  
0  0.057802  0.048005  back-off     0  

[1 rows x 22 columns]
thumbs-up
sad-man
claws
laughing-couple
by-the-sea
arguing
enjoying-the-sun
piggyback
back-off
handshake
tablet
happy-man
sad-woman
business
pain
students
bills
bullied
disagreement
grandparents


100%|██████████| 1/1 [00:02<00:00,  2.22s/it]
C:\Users\Shuai Shao\AppData\Local\Temp\ipykernel_22600\2147124916.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aus['file'] = name
C:\Users\Shuai Shao\AppData\Local\Temp\ipykernel_22600\2147124916.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aus['face'] = range(len(aus))


       AU01      AU02      AU04      AU05      AU06  AU07      AU09      AU10  \
0  0.292465  0.432277  0.175178  0.229799  0.950113   1.0  0.580514  0.990095   

   AU11      AU12  ...      AU17  AU20      AU23      AU24      AU25  \
0   0.0  0.987156  ...  0.125573   1.0  0.045834  0.005221  0.999958   

       AU26     AU28      AU43       file  face  
0  0.329711  0.01957  0.277995  thumbs-up     0  

[1 rows x 22 columns]


100%|██████████| 1/1 [00:02<00:00,  2.09s/it]
C:\Users\Shuai Shao\AppData\Local\Temp\ipykernel_22600\2147124916.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aus['file'] = name
C:\Users\Shuai Shao\AppData\Local\Temp\ipykernel_22600\2147124916.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aus['face'] = range(len(aus))


...

100%|██████████| 1/1 [00:03<00:00,  3.11s/it]
C:\Users\Shuai Shao\AppData\Local\Temp\ipykernel_22600\2147124916.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aus['file'] = name
C:\Users\Shuai Shao\AppData\Local\Temp\ipykernel_22600\2147124916.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aus['face'] = range(len(aus))


       AU01      AU02      AU04      AU05      AU06  AU07      AU09      AU10  \
0  0.348266  0.271851  0.289249  0.577633  0.096787   0.0  0.114005  0.011003   
1  0.302074  0.239038  0.590965  0.216845  0.049979   0.0  0.189569  0.015603   
2  0.458317  0.240075  0.126495  0.295747  0.071853   0.0  0.220087  0.230844   

   AU11      AU12  ...      AU17  AU20      AU23      AU24      AU25  \
0   0.0  0.029945  ...  0.492295   0.0  0.437687  0.482573  0.064133   
1   0.0  0.023931  ...  0.221122   0.0  0.287109  0.180640  0.967365   
2   1.0  0.041507  ...  0.441875   1.0  0.538142  0.149729  0.945268   

       AU26      AU28      AU43          file  face  
0  0.103712  0.255633  0.033056  disagreement     0  
1  0.727386  0.034110  0.128918  disagreement     1  
2  0.493449  0.221965  0.033692  disagreement     2  

[3 rows x 22 columns]


100%|██████████| 1/1 [00:02<00:00,  2.93s/it]

       AU01      AU02      AU04      AU05      AU06  AU07      AU09      AU10  \
0  0.264805  0.322456  0.309800  0.322348  0.634366   1.0  0.667120  0.167038   
1  0.497441  0.268270  0.416279  0.220653  0.914461   1.0  0.773256  0.350736   
2  0.467048  0.095630  0.875151  0.337699  0.092479   0.0  0.324306  0.002855   

   AU11      AU12  ...      AU17  AU20      AU23      AU24      AU25  \
0   0.0  0.785027  ...  0.265023   1.0  0.315057  0.014834  0.999972   
1   0.0  0.945459  ...  0.183925   1.0  0.199229  0.011019  0.999941   
2   0.0  0.013731  ...  0.617463   0.0  0.743830  0.748477  0.469512   

       AU26      AU28      AU43          file  face  
0  0.585865  0.135921  0.916586  grandparents     0  
1  0.804257  0.009645  0.277607  grandparents     1  
2  0.765543  0.327428  0.638592  grandparents     2  

[3 rows x 22 columns]
[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]



C:\Users\Shuai Shao\AppData\Local\Temp\ipykernel_22600\2147124916.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aus['file'] = name
C:\Users\Shuai Shao\AppData\Local\Temp\ipykernel_22600\2147124916.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aus['face'] = range(len(aus))


## Reflections
### 1.


### 2.
* I would use backward feature elimination to determine which features yields the best performance increase / lowest performance decrease at the specified number of features. While it may be tempting to choose the AUs with the greatest mean difference between classes, these AUs may be intercorrelated, meaning that using more than one would may provide little to new information gain.
* Having too many features can be a problem as the solution space becomes much larger and requires more data in order to train the model, otherwise possibly leading to overfitting, known as the curse of dimensionality.